In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

In [ ]:
# programmer - Sophia Quinton
# date - 12-22-21
# class - DSC -540
# assignment - Assignment 8

#acquire data
#(McKinley et. al., 2010) (Harris et. al., 2020)
import pandas as pd
import numpy as np


In [ ]:
#(Urban, 2019)
def make_views(
  arr,
  win_size,
  step_size,
  writeable = False,
):
  """
  arr: any 2D array whose columns are distinct variables and 
    rows are data records at some timestamp t
  win_size: size of data window (given in data points along record/time axis)
  step_size: size of window step (given in data point along record/time axis)
  writable: if True, elements can be modified in new data structure, which will affect
    original array (defaults to False)
  
  Note that step_size is related to window overlap (overlap = win_size - step_size), in 
  case you think in overlaps.
  """
  
  n_records = arr.shape[0]
  n_columns = arr.shape[1]
  remainder = (n_records - win_size) % step_size 
  num_windows = 1 + int((n_records - win_size - remainder) / step_size)
  new_view_structure = np.lib.stride_tricks.as_strided(
    arr,
    shape = (num_windows, win_size, n_columns),
    strides = (8 * step_size * n_columns, 8 * n_columns, 8),
    writeable = False,
  )
  return new_view_structure
  


In [ ]:
#(Chowdhury et. al., 2017)
def spectral(input_column):
  FFT = np.fft.fft(input_column)
  FFT_max = float(max(FFT))
  summation_data =  np.sum(np.square(np.absolute(FFT)))
  final_result = summation_data/1000
  return final_result, FFT_max


In [ ]:
def zeroCross(input):
  ##(Chowdhury et. al., 2017) (Muller, 2015)
  X = input-np.median(input)
  result = ((X[:-1] * X[1:]) < 0).sum()
  return result

In [ ]:
#s_1_cut2
def get_activity(index, full_frame):
  if index != 0:
    new_index = index + 1000
  else:
    new_index = index
  act = np.array(full_frame[1])[new_index]
  return act

In [ ]:
def get_correlate(table_frame, index1, index2, index3):
  k =pd.DataFrame({'x':table_frame[index1][index2], 'y':table_frame[index1][index3]})
  correlate = k.corr()
  out = correlate['x'][1]
  return out


In [ ]:
##get metrics
from scipy.stats import kurtosis, skew
def get_metrics(table_frame, full_frame):
  mean = []
  standard = []
  minimum = []
  maximum = []
  variance = []
  median = []
  skewness = []
  p_25 = []
  p_75 = []
  kurt = []
  spectral_e = []
  cor_axis = []
  zero_cross = []
  dominantF = []
  activity = []
  counter = 0

  for i in range(len(table_frame)):
      mean_group = []
      standard_group = []
      min_group = []
      max_group = []
      var_group = []
      median_group = []
      skewness_group = []
      p25_group = []
      p75_group = []
      kurtosis_group = []
      spectral_e_group = []
      zero_cross_group = []
      dominantF_group = []

      for j in range(3):
          column = [ row[j] for row in table_frame[i]]
          mean_1 = np.mean(column)
          standard_1 = np.std(column)
          min_1 = min(column)
          max_1 = max(column)
          var_1 = np.var(column)
          median_1 = np.median(column)
          skewness_1 = skew(column)
          p25_1 = np.percentile(column, 25)
          p75_1 = np.percentile(column, 75)
          kurtosis_1 = kurtosis(column)
          spectral_e_1, max_FFT_1 = spectral(column)
          zero_cross_1 = zeroCross(column)

          mean_group.append(mean_1)
          standard_group.append(standard_1)
          min_group.append(min_1)
          max_group.append(max_1)
          var_group.append(var_1)
          median_group.append(median_1)
          skewness_group.append(skewness_1)
          p25_group.append(p25_1)
          p75_group.append(p75_1)
          kurtosis_group.append(kurtosis_1)
          spectral_e_group.append(spectral_e_1)
          zero_cross_group.append(zero_cross_1)
          dominantF_group.append(max_FFT_1)

      mean.append(mean_group)
      standard.append(standard_group)
      minimum.append(min_group)
      maximum.append(max_group)
      variance.append(var_group)
      median.append(median_group)
      skewness.append(skewness_group)
      p_25.append(p25_group)
      p_75.append(p75_group)
      kurt.append(kurtosis_group)
      spectral_e.append(spectral_e_group)
      zero_cross.append(zero_cross_group)
      dominantF.append(dominantF_group)

      #correlation tests
      xy = get_correlate(table_frame, i, 0, 1)
      xz = get_correlate(table_frame, i, 0, 2)
      yz = get_correlate(table_frame, i, 1, 2)
      correlation_group = np.array([xy, xz, yz])
      cor_axis.append(correlation_group)

      #activity
      activity_1 = get_activity(counter, full_frame)
      activity.append(activity_1)
      counter += 1

  sub1 = pd.concat([pd.DataFrame(mean),
           pd.DataFrame(standard),
           pd.DataFrame(minimum),
           pd.DataFrame(maximum),
           pd.DataFrame(variance),
           pd.DataFrame(median),
           pd.DataFrame(skewness),
           pd.DataFrame(p_25),
           pd.DataFrame(p_75),
           pd.DataFrame(kurt),
           pd.DataFrame(spectral_e),
           pd.DataFrame(zero_cross),
           pd.DataFrame(dominantF),
           pd.DataFrame(cor_axis),
           pd.DataFrame(activity)], axis = 1)
  return sub1

In [ ]:
#combine the results
def combine_results(current_frame, frame_to_add):
  frame = pd.concat([current_frame, frame_to_add], axis=0)
  return frame

In [ ]:
win = 1000
steps = 500
def all_subjects(path, window_size, step, full_frame):
  s_1 = pd.read_table(path, sep="\s+", header=None)
  s_1_cut = s_1.iloc[:,[0,1,5,6,7]]
  s_1_cut = s_1_cut[s_1_cut[1] < 9]
  #(Agrawal, 2021)
  s_1_cut.interpolate()

  max_1 = max(s_1_cut[0])
  min_1 = min(s_1_cut[0])
  s_1_cut2 = s_1_cut[s_1_cut[0] < (10-max_1)]
  s_1_cut2 = s_1_cut[s_1_cut[0] > (10+min_1)]
  s_1_cut2 = s_1_cut2.set_index(0)
  s_1_cut22 = s_1_cut2[[5,6,7]]
  array_s1 = np.array(s_1_cut22)

  test1 = make_views(array_s1, window_size, step)
  new_frame = get_metrics(test1, s_1_cut2)
  next_frame = combine_results(full_frame, new_frame)
  return next_frame


In [ ]:
full_frame = pd.DataFrame()
s_1 = all_subjects("/content/gdrive/MyDrive/data/subject101.dat", win, steps, full_frame)
s_2 = all_subjects("/content/gdrive/MyDrive/data/subject102.dat", win, steps, s_1)
s_3 = all_subjects("/content/gdrive/MyDrive/data/subject103.dat", win, steps, s_2)
s_4 = all_subjects("/content/gdrive/MyDrive/data/subject104.dat", win, steps, s_3)
s_5 = all_subjects("/content/gdrive/MyDrive/data/subject105.dat", win, steps, s_4)
s_6 = all_subjects("/content/gdrive/MyDrive/data/subject106.dat", win, steps, s_5)
s_7 = all_subjects("/content/gdrive/MyDrive/data/subject107.dat", win, steps, s_6)
s_8 = all_subjects("/content/gdrive/MyDrive/data/subject108.dat", win, steps, s_7)
s_9 = all_subjects("/content/gdrive/MyDrive/data/subject109.dat", win, steps, s_8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to

In [ ]:
s_9.columns = ["meanx", "meany", "meanz",
               "standardx", "standardy", "standardz",
               "minimumx", "minimumy", "minimumz",
               "maximumx", "maximumy", "maximumz",
               "variancex", "variancey", "variancez",
               "medianx", "mediany", "medianz",
               "skewnessx", "skewnessy", "skewnessz",
               "percent_25x", "percent_25y", "percent_25z",
               "percent_75x", "percent_75y", "percent_75z",
               "kurtosisx", "kurtosisy", "kurtosisz",
               "spectralE_x", "spectralE_y", "spectralE_z",
               "zero_crossx", "zero_crossy", "zero_crossz",
               "dominant_freqx", "dominant_freqy", "dominant_freqz",
               "corxy", "corxz", "coryz",
               "activity"]

In [ ]:
s_9.to_csv("/content/gdrive/MyDrive/data/feature_extract.csv")

In [ ]:
s_1 = pd.read_table("/content/gdrive/MyDrive/data/subject101.dat", sep="\s+", header=None)
s_1_cut = s_1.iloc[:,[0,1,2,5,6,7]]
s_1_cut = s_1_cut[s_1_cut[1] < 9]

In [ ]:
s_1_cut[1][5000]

1

In [ ]:
get_activity(5000, s_1_cut)

1

In [ ]:
s_9


,meanx,meany,meanz,standardx,standardy,standardz,minimumx,minimumy,minimumz,maximumx,maximumy,maximumz,variancex,variancey,variancez,medianx,mediany,medianz,skewnessx,skewnessy,skewnessz,percent_25x,percent_25y,percent_25z,percent_75x,percent_75y,percent_75z,kurtosisx,kurtosisy,kurtosisz,spectralE_x,spectralE_y,spectralE_z,zero_crossx,zero_crossy,zero_crossz,dominant_freqx,dominant_freqy,dominant_freqz,corxy,corxz,coryz,activity
0,8.277640,8.275093,8.275220,2.350354,2.364960,2.359422,-6.95140,-7.025440,-7.140180,12.451700,12.412300,11.996800,5.524162,5.593038,5.566872,8.790680,8.794115,8.803195,-3.918387,-3.934200,-3.917623,8.123150,8.121625,8.121455,9.170285,9.173640,9.146230,18.678349,18.753629,18.613747,74043.489536,74070.197967,74046.145945,217,216,200,8277.640231,8275.092743,8275.220458,0.916711,-0.992975,-0.862994,0
1,6.455912,6.451762,6.454657,2.599502,2.601844,2.595849,-6.95140,-7.025440,-7.140180,12.451700,12.412300,11.996800,6.757411,6.769590,6.738434,5.966395,5.992940,5.999215,-1.446654,-1.492945,-1.475961,4.985642,4.986190,4.985480,8.142302,8.146358,8.189435,5.709036,5.919741,5.851275,48436.208363,48394.826496,48401.034164,49,55,61,6455.911840,6451.762290,6454.657237,0.687687,0.091654,-0.659922,0
2,5.150309,5.157433,5.157747,0.670588,0.665078,0.661195,1.05451,0.262818,0.261184,10.664300,9.832780,9.593610,0.449688,0.442328,0.437178,5.024570,5.025640,5.025060,-0.104543,-0.329419,-0.225713,4.911100,4.911817,4.909900,5.390550,5.397607,5.431522,12.087285,10.926299,10.614695,26975.375443,27041.445634,27039.536292,197,206,197,5150.309480,5157.433217,5157.747379,-0.999672,0.976152,-0.981390,0
3,5.052481,5.056136,5.056156,0.111128,0.110796,0.110105,4.68494,4.683310,4.684730,5.331620,5.366050,5.442100,0.012349,0.012276,0.012123,5.063305,5.063665,5.063520,-0.182251,-0.289412,-0.165385,4.987190,4.987840,4.988240,5.138043,5.138790,5.138820,-0.001121,0.103551,0.104629,25539.912735,25576.782760,25576.831896,350,327,315,5052.480900,5056.135570,5056.155540,-0.962861,-0.861402,0.692271,0
4,5.121723,5.121846,5.127689,0.262669,0.262401,0.264596,2.65483,2.461280,2.425850,7.251170,7.104180,7.279790,0.068995,0.068854,0.070011,5.102625,5.102780,5.103230,0.906257,0.519841,0.833106,5.063160,5.062910,5.063450,5.176837,5.177410,5.177980,35.918268,35.615555,37.291367,26301.037106,26302.158690,26363.205280,454,440,394,5121.722580,5121.845790,5127.688980,0.144270,0.397924,0.965229,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,NaN,NaN,NaN,NaN,NaN,NaN,-20.28380,-19.967000,-20.387700,1.187310,1.125900,0.898926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,-0.866111,0.983164,-0.942866,0
635,NaN,NaN,NaN,NaN,NaN,NaN,-20.28380,-19.967000,-20.387700,1.187310,1.125900,0.898926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,1.000000,-1.000000,NaN,0
636,NaN,NaN,NaN,NaN,NaN,NaN,-20.58270,-22.611300,-19.748900,0.128449,0.136433,0.188098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0.906035,0.952587,0.734312,0
637,NaN,NaN,NaN,NaN,NaN,NaN,-22.18000,-22.614600,-21.400200,-3.325980,-3.098010,-3.247570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0.989895,-0.963985,-0.991957,0


In [ ]:
s_2

,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0
0,8.277640,8.275093,8.275220,2.350354,2.364960,2.359422,-6.95140,-7.025440,-7.140180,12.451700,12.412300,11.996800,5.524162,5.593038,5.566872,8.790680,8.794115,8.803195,-3.918387,-3.934200,-3.917623,8.123150,8.121625,8.121455,9.170285,9.173640,9.146230,18.678349,18.753629,18.613747,74043.489536,74070.197967,74046.145945,217,216,200,8277.640231,8275.092743,8275.220458,0.916711,-0.992975,-0.862994,0
1,6.455912,6.451762,6.454657,2.599502,2.601844,2.595849,-6.95140,-7.025440,-7.140180,12.451700,12.412300,11.996800,6.757411,6.769590,6.738434,5.966395,5.992940,5.999215,-1.446654,-1.492945,-1.475961,4.985642,4.986190,4.985480,8.142302,8.146358,8.189435,5.709036,5.919741,5.851275,48436.208363,48394.826496,48401.034164,49,55,61,6455.911840,6451.762290,6454.657237,0.687687,0.091654,-0.659922,0
2,5.150309,5.157433,5.157747,0.670588,0.665078,0.661195,1.05451,0.262818,0.261184,10.664300,9.832780,9.593610,0.449688,0.442328,0.437178,5.024570,5.025640,5.025060,-0.104543,-0.329419,-0.225713,4.911100,4.911817,4.909900,5.390550,5.397607,5.431522,12.087285,10.926299,10.614695,26975.375443,27041.445634,27039.536292,197,206,197,5150.309480,5157.433217,5157.747379,-0.999672,0.976152,-0.981390,0
3,5.052481,5.056136,5.056156,0.111128,0.110796,0.110105,4.68494,4.683310,4.684730,5.331620,5.366050,5.442100,0.012349,0.012276,0.012123,5.063305,5.063665,5.063520,-0.182251,-0.289412,-0.165385,4.987190,4.987840,4.988240,5.138043,5.138790,5.138820,-0.001121,0.103551,0.104629,25539.912735,25576.782760,25576.831896,350,327,315,5052.480900,5056.135570,5056.155540,-0.962861,-0.861402,0.692271,0
4,5.121723,5.121846,5.127689,0.262669,0.262401,0.264596,2.65483,2.461280,2.425850,7.251170,7.104180,7.279790,0.068995,0.068854,0.070011,5.102625,5.102780,5.103230,0.906257,0.519841,0.833106,5.063160,5.062910,5.063450,5.176837,5.177410,5.177980,35.918268,35.615555,37.291367,26301.037106,26302.158690,26363.205280,454,440,394,5121.722580,5121.845790,5127.688980,0.144270,0.397924,0.965229,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,NaN,NaN,NaN,NaN,NaN,NaN,-20.64300,-20.169600,-24.875500,20.242500,21.695300,18.162700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0.963787,-1.000000,-1.000000,0
696,NaN,NaN,NaN,NaN,NaN,NaN,-18.60770,-13.507700,-24.875500,20.242500,21.695300,18.162700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,-0.827635,-0.059198,-0.511288,0
697,NaN,NaN,NaN,NaN,NaN,NaN,-1.55833,-1.502220,-0.974850,0.787792,1.059820,1.665330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,-0.494859,-0.044439,0.890106,0
698,-0.070917,-0.070206,-0.070756,0.026733,0.025924,0.027411,-0.17260,-0.173981,-0.219576,0.024319,0.053072,0.051941,0.000715,0.000672,0.000751,-0.067740,-0.067737,-0.067745,-0.046531,0.159038,-0.193334,-0.083063,-0.082938,-0.083188,-0.052418,-0.052541,-0.052417,0.557122,0.993750,1.643851,5.743825,5.600873,5.757823,498,481,508,2.120973,1.772977,1.674899,-0.160856,1.000000,-0.161117,0
